In [ ]:
# 1.1 Importações necessárias
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Criando um conjunto de dados de exemplo (na prática, usariam o do projeto)
# Dica: Substituam isso pelo código que carrega e pré-processa os dados do projeto de vocês.
corpus = [
    "o filme é excelente e muito divertido",
    "não gostei do enredo, muito chato",
    "a atuação foi brilhante e o roteiro impecável",
    "péssimo! perdi meu tempo assistindo",
    "a história é ok, mas nada de mais",
    "filme horrível, nem vale a pena",
    "uma obra-prima da sétima arte",
    "o filme tem um final surpreendente e emocionante",
    "este é o pior filme que já vi na vida"
]

labels = ["positivo", "negativo", "positivo", "negativo", "neutro", "negativo", "positivo", "positivo", "negativo"]

# Separando dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(corpus, labels, test_size=0.3, random_state=42)

# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_treino_vetorizado = vectorizer.fit_transform(X_treino)
X_teste_vetorizado = vectorizer.transform(X_teste)

In [ ]:
# 1.2 Criando e treinando o modelo inicial (com parâmetros padrão)
modelo_base = SVC()
modelo_base.fit(X_treino_vetorizado, y_treino)

# Fazendo previsões no conjunto de teste
previsoes = modelo_base.predict(X_teste_vetorizado)

# Avaliando o modelo inicial
print("### Análise Inicial do Modelo (linha de base) ###\n")
print("Relatório de Classificação:")
print(classification_report(y_teste, previsoes, zero_division=0))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_teste, previsoes))

### Análise Inicial do Modelo (linha de base) ###

Relatório de Classificação:
              precision    recall  f1-score   support

    negativo       0.00      0.00      0.00         2
    positivo       0.33      1.00      0.50         1

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3


Matriz de Confusão:
[[0 2]
 [0 1]]


Analise os Resultados:

A baixa performance do modelo (acurácia e F1-score baixos) neste primeiro passo é causada, primariamente, pela qualidade e tamanho insuficientes dos dados
 F1-score para classes com suporte baixíssimo é não confiável.O modelo não consegue performar bem porque a qualidade dos dados (neste caso, a falta deles) compromete a capacidade de aprendizado, independentemente do algoritmo escolhido5.
 2. Matriz de Confusão: Onde o Modelo Erra e o que Ele AprendeuAo examinar a matriz de confusão, o padrão de erro provável nos diz que o modelo "aprendeu" muito pouco ou nada de útil:Classe com Maior Erro: É altamente provável que o modelo esteja errando a maioria das previsões na classe com menor número de amostras (suporte), que é a classe "neutro" (apenas 1 amostra no corpus original 6). O modelo simplesmente não viu exemplos suficientes do que é um sentimento neutro para classificar corretamente.
 O que o Modelo "Aprendeu": A matriz de confusão provavelmente mostrará que o modelo tende a classificar quase tudo na classe majoritária ou naquela que ele "aprendeu" mais facilmente. Isso significa que ele não aprendeu a distinção fina entre as classes; ele apenas se tornou bom em chutar a categoria mais frequente no conjunto de treino.Em suma, a matriz de confusão reforça que o modelo está superajustado (overfitting) aos poucos dados ou está subajustado (underfitting), falhando em capturar qualquer padrão significativo.

In [ ]:
# Avaliando o modelo com validação cruzada
# Usamos o modelo base novamente para a avaliação
scores = cross_val_score(modelo_base, X_treino_vetorizado, y_treino, cv=3, scoring='f1_macro')

print("\n### Avaliação com Validação Cruzada ###\n")
print(f"Scores para cada 'fold': {scores}")
print(f"Média do F1-score com Validação Cruzada: {scores.mean():.2f}")


### Avaliação com Validação Cruzada ###

Scores para cada 'fold': [0.33333333 0.33333333 0.33333333]
Média do F1-score com Validação Cruzada: 0.33


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [ ]:
# Definindo os hiperparâmetros que queremos testar
# 'C' controla a margem de classificação (trade-off entre regularização e erro de treino)
# 'kernel' define a função de kernel usada pelo SVM
parametros = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

# Criando o objeto GridSearchCV
# cv=3 significa que testaremos cada combinação 3 vezes
grid_search = GridSearchCV(SVC(), parametros, cv=3, scoring='f1_macro')

# Executando a busca na "grade"
print("\n### Buscando os Melhores Hiperparâmetros... ###")
grid_search.fit(X_treino_vetorizado, y_treino)

# Exibindo os melhores resultados
print("\nBusca Concluída!")
print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
print(f"Melhor pontuação (F1-score) encontrada: {grid_search.best_score_:.2f}")

# Avaliando o modelo otimizado no conjunto de teste
modelo_otimizado = grid_search.best_estimator_
previsoes_otimizadas = modelo_otimizado.predict(X_teste_vetorizado)

print("\n### Análise do Modelo Otimizado (comparação) ###\n")
print("Relatório de Classificação do Modelo Otimizado:")
print(classification_report(y_teste, previsoes_otimizadas, zero_division=0))
print("\nMatriz de Confusão do Modelo Otimizado:")
print(confusion_matrix(y_teste, previsoes_otimizadas))


### Buscando os Melhores Hiperparâmetros... ###


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



Busca Concluída!
Melhores parâmetros encontrados: {'C': 0.1, 'kernel': 'linear'}
Melhor pontuação (F1-score) encontrada: 0.33

### Análise do Modelo Otimizado (comparação) ###

Relatório de Classificação do Modelo Otimizado:
              precision    recall  f1-score   support

    negativo       0.00      0.00      0.00         2
    positivo       0.33      1.00      0.50         1

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3


Matriz de Confusão do Modelo Otimizado:
[[0 2]
 [0 1]]


Analise dos resultados: A pontuação média da Validação Cruzada ($\text{F1-score}$) pode ser próxima ou ligeiramente diferente da acurácia/F1-score inicial.

In [ ]:
# Importações necessárias
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Criando um dataset de exemplo maior e mais balanceado
# Note que a coleta de dados de um projeto real simula um volume de dados muito maior, para permitir a otimização
corpus = [
    "o filme é excelente e muito divertido", "não gostei do enredo",
    "a atuação foi brilhante", "péssimo! perdi meu tempo assistindo",
    "a história é ok, mas nada de mais", "filme horrível, nem vale a pena",
    "uma obra-prima da sétima arte", "o filme tem um final surpreendente e emocionante",
    "este é o pior filme que já vi na vida", "excelente direção e fotografia",
    "não recomendo, um lixo", "filme maravilhoso, amei", "sem graça e previsível",
    "simplesmente incrível, o melhor filme do ano", "que filme ruim, nao entendi a hype",
    "espetacular, recomendo a todos"
] * 20 # Aumentando o dataset para 320 amostras

labels = (["positivo", "negativo", "positivo", "negativo", "neutro", "negativo", "positivo", "positivo", "negativo", "positivo", "negativo", "positivo", "negativo", "positivo", "negativo", "positivo"] * 20)

# Separando dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(corpus, labels, test_size=0.2, random_state=42, stratify=labels)

# Vetorização dos textos usando TF-IDF com n-grams
# Incluindo 'ngram_range' para capturar frases de 1 e 2 palavras
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=500)
X_treino_vetorizado = vectorizer.fit_transform(X_treino)
X_teste_vetorizado = vectorizer.transform(X_teste)

A perfeição imediata prova o ponto central do roteiro : a qualidade dos dados é tão importante (ou mais) quanto o próprio algoritmo. O trabalho árduo na coleta, balanceamento e engenharia de features (n-grams) elimina a necessidade de otimização complexa inicial.

In [ ]:
# Criando e treinando o modelo inicial (com parâmetros padrão)
modelo_base = SVC()
modelo_base.fit(X_treino_vetorizado, y_treino)

# Fazendo previsões no conjunto de teste
previsoes = modelo_base.predict(X_teste_vetorizado)

# Avaliando o modelo inicial
print("### Análise Inicial do Modelo (linha de base) ###\n")
print("Relatório de Classificação:")
print(classification_report(y_teste, previsoes, zero_division=0))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_teste, previsoes))

### Análise Inicial do Modelo (linha de base) ###

Relatório de Classificação:
              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00        28
      neutro       1.00      1.00      1.00         4
    positivo       1.00      1.00      1.00        32

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64


Matriz de Confusão:
[[28  0  0]
 [ 0  4  0]
 [ 0  0 32]]


In [ ]:
# Avaliando o modelo com validação cruzada
# Usamos o modelo base novamente para a avaliação
scores = cross_val_score(modelo_base, X_treino_vetorizado, y_treino, cv=5, scoring='f1_macro')

print("\n### Avaliação com Validação Cruzada ###\n")
print(f"Scores para cada 'fold': {scores.round(2)}")
print(f"Média do F1-score com Validação Cruzada: {scores.mean():.2f}")


### Avaliação com Validação Cruzada ###

Scores para cada 'fold': [1. 1. 1. 1. 1.]
Média do F1-score com Validação Cruzada: 1.00


Validação da Generalização: No Exemplo 1, scores inconsistentes (ex: $0.20, 0.80$) indicavam instabilidade. Neste caso, scores uniformes provam que o modelo não está superajustado (overfitting) a uma única divisão específica dos dados, mas sim que aprendeu padrões de sentimento verdadeiramente generalizáveis2.Confirmação do Potencial do Dataset: A Validação Cruzada confirma que o esforço em aumentar e balancear o dataset, juntamente com o pré-processamento (n-grams), foi a chave para construir um modelo confiável e estável. O modelo é robusto porque a qualidade da sua base de aprendizado é alta.

In [ ]:
# Definindo os hiperparâmetros que queremos testar
# 'C' controla a regularização e 'gamma' afeta a "curvatura" da fronteira de decisão
parametros = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Criando o objeto GridSearchCV
# cv=5 para uma avaliação mais robusta
grid_search = GridSearchCV(SVC(), parametros, cv=5, scoring='f1_macro')

# Executando a busca na "grade"
print("\n### Buscando os Melhores Hiperparâmetros... ###")
grid_search.fit(X_treino_vetorizado, y_treino)

# Exibindo os melhores resultados
print("\nBusca Concluída!")
print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
print(f"Melhor pontuação (F1-score) encontrada: {grid_search.best_score_:.2f}")

# Avaliando o modelo otimizado no conjunto de teste
modelo_otimizado = grid_search.best_estimator_
previsoes_otimizadas = modelo_otimizado.predict(X_teste_vetorizado)

print("\n### Análise do Modelo Otimizado (comparação) ###\n")
print("Relatório de Classificação do Modelo Otimizado:")
print(classification_report(y_teste, previsoes_otimizadas, zero_division=0))
print("\nMatriz de Confusão do Modelo Otimizado:")
print(confusion_matrix(y_teste, previsoes_otimizadas))


### Buscando os Melhores Hiperparâmetros... ###

Busca Concluída!
Melhores parâmetros encontrados: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Melhor pontuação (F1-score) encontrada: 1.00

### Análise do Modelo Otimizado (comparação) ###

Relatório de Classificação do Modelo Otimizado:
              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00        28
      neutro       1.00      1.00      1.00         4
    positivo       1.00      1.00      1.00        32

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64


Matriz de Confusão do Modelo Otimizado:
[[28  0  0]
 [ 0  4  0]
 [ 0  0 32]]


Ponto de Saturação: Quando um modelo já atinge uma performance muito próxima de 100%, há pouco ou nenhum espaço para melhoria. O algoritmo SVC base, alimentado com dados de alta qualidade e n-grams, já encontrou uma fronteira de decisão que separa as classes com altíssima precisão.Ajuste Mínimo: Se houver alguma mudança nos resultados, ela será minúscula (ex: F1-score de $0.99$ para $0.995$) ou pode até mesmo ser uma pequena queda, indicando que a otimização pode ter tentado ajustar o modelo demais (leve overfitting) ao conjunto de treino.Qual é o Valor da Otimização Neste Caso?O valor da otimização com Grid Search (buscar os melhores parâmetros C, kernel, e gamma) neste cenário não está em aumentar a pontuação, mas sim em confirmar a estabilidade e a robustez do modelo.

Reflexão Final: O Poder da Qualidade dos Dados
1. Qual Foi a Principal Lição que Você Tirou Desses Dois Cenários?
A principal lição é: A Qualidade e a Quantidade dos Dados Superam a Otimização do Algoritmo.